In [23]:
from helper import reverse_dictionary

SNAFU = {
    "=": -2,
    "-": -1,
    "2": 2,
    "1": 1,
    "0": 0
}

def SNAFU_Parser(SNAFU_input:str) -> int:
    return sum(SNAFU[i]*5**exp for exp, i in enumerate(SNAFU_input[::-1]))   


def inverse_SNAFU_Parser(decimal_input:int, position: int) -> str:
    return SNAFU_Inversed[decimal_input%(5**position)]

# SNAFU_Inversed = reverse_dictionary(SNAFU)
# Invert_SNAFU = lambda num, pos  : SNAFU_Inversed[int(num)/(5**pos)]
# concat_SNAFU = lambda charac, global_str: global_str.append(charac)

 
global_str = ""
for pos, num in enumerate(str(1747)):
    global_str+inverse_SNAFU_Parser(int(num), pos)
    

# sum(SNAFU_Parser(requirement) for requirement in open("day25.txt").read().split("\n"))
# inverse_SNAFU_Parser(1747)
print(global_str)

KeyError: 4

In [ ]:
from itertools import product
from re import findall
cartesian_product = lambda vector1, vector2: product(vector1,vector2)
data = [tuple(map(int, findall("\d+", line))) for line in open("day18_test.txt").read().split("\n")]

# sorted(list(filter(lambda d: len(set(d))<3, data)))
# [3-len(set(d)) for d in data]
# sorted(data)
distance = lambda tuples: [x-y for x,y in zip(*tuples)]
# filteration = lambda tuples: tuples[0] != tuples[1]
# list(filter(filteration,cartesian_product(data, data)))
check_the_touch = lambda lst: all([abs(x)<2 for x in lst])
sorted(y for y in [x for x in [distance(i) for i in cartesian_product(data, data)] if x.count(0) == 2] if check_the_touch(y))

In [ ]:
# DAY 17
# >>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>
import re
from collections import deque
CHAMBER_WIDTH = 7
APPEAR_PADDING = 2
DIST_BETWEEN_ROCKS = 3
direction_parse = lambda direction_char: -1 if direction_char=="<" else 1
DIRECTION_DATA = deque(map(direction_parse, open("day17_test.txt").read().strip()))


rocks = """####

.#.
###
.#.

..#
..#
###

#
#
#
#

##
##"""
shapes = "-+⅃|□"
height = [len(re.findall("\n", rock))+1 for rock in rocks.split("\n\n")]
width = (4,3,3,1,2)
top_spans = (lambda x_span: x_span, lambda x_span: (x_span[0]+1, x_span[0]-1), lambda x_span: x_span, lambda x_span: x_span)
rock_profile = deque([{"shape": shape, "height": height, "width": width, "position": {"x_span": (APPEAR_PADDING, APPEAR_PADDING+width-1), "y_span": (0,height), "top_span": top_span}} for shape, height, width, top_span in zip(shapes, height, width, top_spans)]) 



def mover() -> None:
    rock = rock_profile[0]
    left, right, top, bottom = (rock["position"]["x_span"][0], rock["position"]["x_span"][1], rock["position"]["y_span"][0], rock["position"]["y_span"][1])
    print(left, right, top, bottom)      
    vertical_movement_allowed = True
    previous_rock_span = (0,7)    
    while(vertical_movement_allowed):      
        direction =  DIRECTION_DATA[0]
        print(direction)
        left, right, top, bottom = (rock["position"]["x_span"][0]+direction, rock["position"]["x_span"][1]+direction, rock["position"]["y_span"][0]+1, rock["position"]["y_span"][1]+1)
        print(left, right, top, bottom)        
        horizontal_movement_allowed=True if left>0 and right<CHAMBER_WIDTH else False
        vertical_movement_allowed = True if DIST_BETWEEN_ROCKS-top>0 or (top==3 and any(range(left, right-1)) not in range(*previous_rock_span)) else False
        rock["position"].update({"y_span": (top,bottom)})            
        if horizontal_movement_allowed:
            rock["position"].update({"x_span": (left,right)})
        else: print("horizontal didnt happen")                  
        print(rock["position"])
        DIRECTION_DATA.rotate(-1)
        previous_rock_span = rock["position"]["top_span"](rock["position"]["x_span"])
        print(f"Prev rock span {previous_rock_span}")
    print("****************************************************************")          
    rock_profile.rotate(-1)
    print(f"Next rock: {rock_profile[0]['shape']}")
    

def main():
    mover(rock_profile[0])
    

print(rock_profile)    
mover()
mover()


In [ ]:
# DAY 16
import re
# 💀 DANGER 💀: Crazy regex stuff ahead : [A-Z]{2} matches pattern like AA and \d+ matches integers
[re.findall(r'[A-Z]{2}|\d+', line) for line in open("day16_test.txt").read().split("\n")]


In [ ]:
data = [[eval(message) for message in pair.split("\n")] for pair in open("day13_test.txt").read().split("\n\n")]
test = list(zip(data[7][0],data[7][1]))
for left, right in test:
    count = 0
    if (type(left) == type(right) and isinstance(left, int)):
        if left<=right:
           continue            
        else: count+=1
    elif type(left) != type(right):
         test = [list(left), right] if isinstance(left, int) else [left, list(right)]
    else: test = zip(left, right)

# cases
# both are int → easy
# both are list → call zip (left, right)
# one is int other is list → Make int into list



In [ ]:
def parse(data):
    directions = {"R": 1 + 0j, "L": -1 + 0, "U": 0 + 1j, "D": 0 - 1j}
    for line in data.splitlines():
        direction, steps = line.split()
        yield directions[direction], int(steps) # R 4 -> ((1+0j),4). Move 4 directions in real axis.


def movement(rope, direction, steps, visited=set()):
    for _ in range(steps):
        rope[0] += direction
        for i in range(len(rope) - 1):
            head, tail = rope[i], rope[i + 1]
            diff = tail - head
            if abs(diff) >= 2:
                if diff.real != 0:
                    tail = tail.real - (diff.real / abs(diff.real)) + tail.imag * 1j
                if diff.imag != 0:
                    tail = tail.real + (tail.imag - (diff.imag / abs(diff.imag))) * 1j
                
            rope[i + 1] = tail
        visited.add(rope[-1])
    return rope


def wiggle(data, rope_length):
    rope = [0j] * rope_length
    visited = set([0])
    for direction, steps in data:
        rope = movement(rope, direction, steps, visited)
        visited.add(rope[-1])
    return visited


def main(day, part=1):
    day.data = parse(day.data)
    if part == 1:
        return len(wiggle(day.data, rope_length=2))
    if part == 2:
        return len(wiggle(day.data, rope_length=10))

data = """R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2"""


    # p1 = main(day)
    # print(p1)


#     data = """R 5
# U 8
# L 8
# D 3
# R 17
# D 10
# L 25
# U 20"""

#     # day.load(data, typing=str)
#     day.load(typing=str)

#     p2 = main(day, part=2)
#     print(p2)
# next(parse(data))
next(parse(open("day9_test.txt").read()))
set([0])

In [ ]:
# from typing import List
# from dataclasses import dataclass

# @dataclass
# class File:
#     name: str
#     size: int

# @dataclass
# class Dir:
#     name: str
#     size: int
#     parent: str
    
# First find all the directories
# Create a hashmap of every directory and subdirectories. Directory: Subdirectories. Look inside every cd
# Start looping. For every subdirectory, look inside it, 
#   case: no directories

    
    

    
# adad.append(csd) 





# def build_fs(term_lines: List[List[str]]):
#     root: Dir = Dir('/', None, [], [])
#     current: Dir = root
#     num_lines = len(term_lines)

In [ ]:
from helper import timeit
class Communication:
    def __init__(self, testpath, path) -> None:
        self.datastream = open(path).read()
        self.teststream = open(testpath).read()
        self.check_repeatition = lambda stream, distinct: len(set(stream)) == distinct
        self.test_cases(1)
        self.test_cases(2)          
        print(f"The answer to Day 5 Part 1 is {self.main(self.datastream, 1)}")
        print(f"The answer to Day 5 Part 2 is {self.main(self.datastream, 2)}") 
   
    @timeit
    def main (self, message, part):
        match part:
            case 1:
                distinct = 4
            case 2:
                distinct = 14
        i = 0        
        j = distinct
        is_marker = False
        while  (j<len(message) and not is_marker):                              
            is_marker = self.check_repeatition(message[i:j], distinct)
            if is_marker:                
                return j
            else:
                i+=1
                j+=1

    def test_cases (self, part):
        match part:
            case 1:
                test_answers = [5,6,10,11]            
            case 2:
                test_answers = [23,23,29,26]
                
        for line, answer in zip(self.teststream.split("\n"), test_answers):       
            assert self.main(line,part) == answer     
       
Communication("day6_test.txt", "day6.txt")
   

In [ ]:
from collections import deque
import re
from helper import timeit
import itertools
from copy import deepcopy

class SupplyStacks:
    def __init__(self, path):
        self.Stacks, self.Instructions = re.split("\n\n", open(path).read())
        self.CARGO = {}
        for i in range(1,max(map(int, re.findall("\d+", self.Stacks)))+1):
            self.CARGO[i] = deque()
          
        for s in re.split("\n", self.Stacks):
            # 💀 DANGER 💀: Some crazy regex stuff ahead
            # "\s{4}|\w" means 4 whitespaces or a word in regex lingo
            for stack, crate in enumerate(re.findall("\s{4}|\w", s)):
                crates = re.findall("[A-Z]+", crate)          
                len(crates) and self.CARGO[stack+1].append(crates[0])
        print(f"The answer to Day 5 Part 1 is {self.simulate_crane(1, deepcopy(self.CARGO))}")
        print(f"The answer to Day 5 Part 2 is {self.simulate_crane(2, self.CARGO)}")
          
    @timeit       
    def simulate_crane(self, part, cargo):        
        match part:
            case 1:     
                for line in self.Instructions.splitlines():            
                    N, From, To = map(int, re.findall("\d+",line))        
                    for i in range(N):                        
                         cargo[To].appendleft(cargo[From].popleft())
                message = ""
                for stack in cargo:
                    message += cargo[stack][0]                    
                return message
            
            case 2:
                for line in self.Instructions.splitlines():            
                    N, From, To = map(int, re.findall("\d+",line))
                    cargo[To].extendleft(list(itertools.islice(cargo[From], 0, N))[::-1])             
                    for i in range(N):                        
                        cargo[From].popleft()
                message = ""
                for stack in cargo:
                    message += cargo[stack][0]
                return message          
 

SupplyStacks("day5.txt")



In [ ]:
from helper import timeit
import re
class CampCleanup:   
    @staticmethod    
    def READ_TEXT_FILE(path):
        for line in open(path):
            # 🤵 Some fancy regex stuff
            yield map(int, re.findall ("\d+", line))

    def __init__(self, path):
        self.path = path
        print(f"The answer to Day 4 Part 1 is {self.main_part1()}")
        print(f"The answer to Day 4 Part 2 is {self.main_part2()}")        
    
    @timeit
    def main_part1(self):
        count  = 0
        for a,b,c,d in self.READ_TEXT_FILE(self.path):
            # 💀DANGER 💀 MINDFUCK LINE AHEAD!🤯 Too much logic stuffed into one line!
            # Get the maximum start element and the minimum end element from both sections
            # Create a tuple of min and max and check if they match any of the sections.
            if (max(a, c),min(b, d)) in [(a,b), (c,d)]: count += 1                   
        return count
    
    @timeit
    def main_part2(self):
        count = 0
        for a,b,c,d in self.READ_TEXT_FILE(self.path):
            if not (c>b or d<a):
               count += 1 
        return count 
    
        
CampCleanup("day4.txt")




In [ ]:
from helper import timeit
import string
class RuckSack:   
    @staticmethod    
    def READ_TEXT_FILE(path):
        return tuple(x for x in open(path).read().split('\n'))
    
    @staticmethod
    def CharacterHashMap() -> dict:
        CharHashMap = {}        
        for index, char in zip(range(1,53),list(string.ascii_lowercase)+list(string.ascii_uppercase)):
            CharHashMap[char] = index
        return CharHashMap     
  
    def __init__(self, path):
        self.DATA = self.READ_TEXT_FILE(path)
        self.CHAR_HASH_MAP = self.CharacterHashMap()
        print(f"The answer to Day 3 Part 1 is {self.main_DAY1()}")
        print(f"The answer to Day 3 Part 2 is {self.main_DAY2()}")
        
        
    def MidpointBreakString (self, string: str) -> int:
        midpoint = int(len(string)/2)
        return [list(string)[:midpoint], list(string)[midpoint:]]
         
    def GetIntersectionCharacter(self, StringList: list) -> str:
        # 💀 DANGER 💀: Verbose Line Ahead
        # I am just unpacking the list of strings and then taking the intersection of all and then re-parsing them into lest to get the common element as string.
        common_character = list(set(StringList[0]).intersection(*StringList))[0]
        return self.CHAR_HASH_MAP[common_character]
    
    @timeit    
    def main_DAY1(self):
        GrandSum = 0
        for sentence in self.DATA:
            Sum = 0            
            Sum += self.GetIntersectionCharacter(self.MidpointBreakString(sentence))
            GrandSum += Sum
        return GrandSum
    
    def GetGroup(self):
        Group = []
        for index, sentence in enumerate(self.DATA):                      
            Group.append(list(sentence))
            if (index+1)%3 == 0:
                yield Group
                Group = []                  
            else:
                continue
            
    @timeit   
    def main_DAY2 (self):
        GrandSum = 0
        
        for group in self.GetGroup():
            Sum = 0
            Sum += self.GetIntersectionCharacter(group)
            GrandSum += Sum
        return GrandSum               
    
RuckSack("day3.txt")



In [ ]:
from helper import timeit

@timeit
def Day2():
    read_text_file = lambda path: tuple(x for x in open(path).read().split('\n'))
    OutcomeSet = [tuple(strng.split()) for strng in read_text_file("day2.txt")]
    score_dictionary = {     
        'X': 1,
        'Y': 2,
        'Z': 3,
        'WinSet': [('A', 'Y'), ('B','Z'), ('C', 'X')],
        'DrawSet': [('A', 'X'), ('B','Y'), ('C', 'Z')],
        'LossSet': [('A', 'Z'), ('B','X'), ('C', 'Y')],      
        'Win': 6,
        'Draw': 3,
        'Loss': 0  
    }

    GrandScore_P1=0
    GrandScore_P2=0
    
    for strategy in OutcomeSet:
        ScoreSum = 0    
        ScoreSum = score_dictionary[strategy[1]]
        if strategy in score_dictionary['WinSet']:
            ScoreSum += score_dictionary['Win']
        elif strategy in score_dictionary['DrawSet']:
            ScoreSum += score_dictionary['Draw']
        else:
            ScoreSum += score_dictionary['Loss']
        GrandScore_P1 += ScoreSum
        
    #Part 2
    for strategy in OutcomeSet:
        ScoreSum = 0
        match strategy[1]:
            case 'X':
                ScoreSum = score_dictionary['Loss']
                # 💀 Danger 💀: Verbose line below. I am just filtering the strategy from Loss Set which matches Elf's strategy and then retrieving the score.            
                ScoreSum += score_dictionary[tuple(x for x in score_dictionary['LossSet'] if x[0] == strategy[0])[0][1]]
            case 'Y':
                ScoreSum = score_dictionary['Draw'] 
                ScoreSum += score_dictionary[tuple(x for x in score_dictionary['DrawSet'] if x[0] == strategy[0])[0][1]]
            case 'Z':
                ScoreSum = score_dictionary['Win'] 
                ScoreSum += score_dictionary[tuple(x for x in score_dictionary['WinSet'] if x[0] == strategy[0])[0][1]]
        GrandScore_P2 += ScoreSum        
        
    return GrandScore_P1, GrandScore_P2
print (f"The answer for Day1 Part1 is {Day2()[0]}")
print (f"The answer for Day1 Part2 is {Day2()[1]}") 


   



In [ ]:
# DAY 1

from helper import timeit
from heapq import heappush, heapify, heappop
read_text_file = lambda path: [x for x in open(path).read().split('\n')]
calorie_list = tuple(read_text_file("C:\git\playground\day1.txt"))
@timeit
def Day1 ():
    calorie_sum= 0  
    heap = []
    heapify(heap)    
    for snack in calorie_list:
        if snack != '':
            calorie_sum += int(snack)            
        else:
            heappush(heap, calorie_sum*-1)
            calorie_sum = 0
    
    max_list = [heappop(heap)*-1,heappop(heap)*-1,heappop(heap)*-1]
    print(f"The solution to Day1 Part1 is {max_list[0]}")
    print(f"The solution to Day1 Part2 is {sum(max_list)}")            
Day1()


